In [ ]:
import subprocess
import json
import pandas as pd

offered_feeds = pd.read_csv('statsperform_feeds.csv')

{
    k:v for k, v in zip(offered_feeds.FeedCode, offered_feeds.FeedWidgetName)
}

In [ ]:
def _access_statsperform_api(feed_name,
                             query_paramo: dict = {
                                 '_rt':'b',
                                 '_fmt':'json',
                             }):

    proxy_url = "http://127.0.0.1:3128"
    statsperform_base_url = 'https://api.performfeeds.com/soccerdata'
    auth_key = ''

    query_string = '&'.join([f'{k}={v}' for k, v in query_paramo.items()])

    print(feed_name)
    if feed_name == 'tournamentcalendar':
        q_command = f""" curl -x "{proxy_url}" '{statsperform_base_url}/{feed_name}/{auth_key}/authorized?&{query_string}' """
    else:
        q_command = f""" curl -x "{proxy_url}" '{statsperform_base_url}/{feed_name}/{auth_key}?&{query_string}' """

    print(q_command)

    process = subprocess.run(
                q_command,
                shell=True,
                capture_output=True,
                check=False # Set to True if you want a CalledProcessError for non-zero exit codes
            )

    stdout = process.stdout
    json_output = json.loads(stdout)
    return json_output


## tournaments

In [ ]:
# Get all available tournament calendar IDs with OT2 feed
feed = 'tournamentcalendar'
comps = _access_statsperform_api(feed)
competitions = pd.DataFrame(comps['competition'])
pd.DataFrame(competitions.iloc[15:20].tournamentCalendar.iloc[0])

In [ ]:
comp_names_ids = pd.DataFrame(competitions.tournamentCalendar.explode().apply(lambda x: {'name':x['name'],
                                                           'id':x['id']}).tolist())


competitions = competitions.explode('tournamentCalendar')
competitions[['competition_name', 'competition_id']] = comp_names_ids



In [ ]:
competitions.name.unique()

## matches

In [ ]:
# Get all matches with MA1 feed

french_matches = _access_statsperform_api(feed_name='match',
                         query_paramo={'_rt':'b',
                          '_fmt':'json',
                          'tmcl':'a7htj8rtzib7a2xx7b3xs04d0',
                          '_pgSz':1000})

all_matches = pd.DataFrame([x['matchInfo'] for x in french_matches['match']])
all_matches

In [ ]:
# getting access to player data using MA2 feed

example_match = '7rd4o9m0otkg7u58cqplq19uc'

example_match_request = _access_statsperform_api(feed_name='matchstats',
                         query_paramo={'_rt':'b',
                          '_fmt':'json',
                          'fx': example_match,
                          # '_pgSz':1000,
                          'detailed':'yes',
                          'people':'yes'})

match_data = pd.DataFrame(example_match_request['liveData']['lineUp'])

dembele = pd.DataFrame(match_data.iloc[1].player).iloc[13]

pd.Series({x['type']:x['value'] for x in dembele.stat})

In [ ]:
example_match_request = _access_statsperform_api(feed_name='rankings',
                         query_paramo={'_rt':'b',
                          '_fmt':'json',
                          'tmcl':'a7htj8rtzib7a2xx7b3xs04d0'})

team_stats = pd.DataFrame(example_match_request['team'])

rabiot = pd.DataFrame(team_stats.iloc[3]['player']).iloc[2]

pd.Series({x['type']:x['value'] for x in rabiot.stat})